In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [2]:
from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string
from tools.notebooks import env

context = Context()

2024-01-08 15:13:35,580 - WARNING - Structure definition for stakings_boosted not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-testnet/deploy_structure.json
2024-01-08 15:13:35,581 - WARNING - Structure definition for metastakings_boosted not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-testnet/deploy_structure.json
2024-01-08 15:13:35,582 - WARNING - Structure definition for position_creator not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-testnet/deploy_structure.json


loaded 10 accounts from 1 PEM files.


2024-01-08 15:13:36,517 - DEBUG - Account.sync_nonce() done: 332
2024-01-08 15:13:36,899 - DEBUG - Account.sync_nonce() done: 3
2024-01-08 15:13:36,908 - INFO - Loaded 5 tokens.
2024-01-08 15:13:37,774 - DEBUG - Account.sync_nonce() done: 332
2024-01-08 15:13:37,782 - INFO - Loaded 3 egld_wraps.
2024-01-08 15:13:37,783 - INFO - Loaded 1 locked_assets.
2024-01-08 15:13:37,784 - INFO - Loaded 1 proxies.
2024-01-08 15:13:37,785 - INFO - Loaded 1 simple_locks.
2024-01-08 15:13:37,787 - INFO - Loaded 1 simple_locks_energy.
2024-01-08 15:13:37,788 - INFO - Loaded 1 fees_collectors.
2024-01-08 15:13:37,789 - INFO - Loaded 1 unstakers.
2024-01-08 15:13:37,790 - INFO - Loaded 1 proxies_v2.
2024-01-08 15:13:37,790 - INFO - Loaded 0 router.
2024-01-08 15:13:37,791 - INFO - Loaded 1 router_v2.
2024-01-08 15:13:37,792 - INFO - Loaded 0 pairs.
2024-01-08 15:13:37,792 - INFO - Loaded 4 pairs_v2.
2024-01-08 15:13:37,793 - INFO - Loaded 1 pairs_view.
2024-01-08 15:13:37,794 - INFO - Loaded 0 farms_comm

In [3]:
from contracts.staking_contract import StakingContract, EnterFarmEvent, ExitFarmEvent, ClaimRewardsFarmEvent
from utils.contract_data_fetchers import StakingContractDataFetcher
staking_contract: StakingContract
staking_contract = context.get_contracts(config.STAKINGS_V2)[0]

In [4]:
from trackers.staking_economics_tracking import StakingEconomics

staking_tracker = StakingEconomics(staking_contract.address, context.network_provider)

ErrBadPubkeyLength: Bad pubkey length: actual = 62, expected = 32

In [ ]:
from contracts.metastaking_contract import MetaStakingContract, EnterMetastakeEvent, ExitMetastakeEvent, ClaimRewardsMetastakeEvent
from utils.contract_data_fetchers import MetaStakingContractDataFetcher
metastaking_contract: MetaStakingContract
metastaking_contract = context.get_contracts(config.METASTAKINGS_BOOSTED)[0]

In [ ]:
ride_purse = Address('')
depositer = Account(pem_file=env.USER1_PEM)
depositer.address = ride_purse
depositer.sync_nonce(context.network_provider.proxy)

upgrade

In [4]:
from contracts.staking_contract import StakingContractVersion
# upgrade all farms
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[StakingContract] = context.get_contracts(config.STAKINGS_V2)
for contract in contracts:
    contract.version = StakingContractVersion.V2
    tx_hash = contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                         config.STAKING_V3_BYTECODE_PATH, [], no_init=True)
    context.network_provider.check_complex_tx_status(tx_hash, "staking upgrade")

2024-01-08 15:15:14,605 - DEBUG - Account.sync_nonce() done: 332
2024-01-08 15:15:14,605 - INFO - Upgrade staking contract
2024-01-08 15:15:14,606 - DEBUG - Upgrade StakingContract contract
2024-01-08 15:15:14,824 - DEBUG - Upgrade arguments: []
2024-01-08 15:15:15,570 - INFO - View this transaction in the MultiversX Testnet Explorer: https://testnet-explorer.multiversx.com/transactions/c0e145f8e648e029c9c85f345b5384d07bf0fffc5928e9d2e70a2e7910174896
2024-01-08 15:15:15,571 - DEBUG - Waiting for transaction c0e145f8e648e029c9c85f345b5384d07bf0fffc5928e9d2e70a2e7910174896 to be executed...
2024-01-08 15:15:22,236 - DEBUG - Transaction c0e145f8e648e029c9c85f345b5384d07bf0fffc5928e9d2e70a2e7910174896 status: success
2024-01-08 15:15:22,236 - DEBUG - TX status most likely false success, making sure...
2024-01-08 15:15:28,805 - DEBUG - Making sure tx status is not false success...
2024-01-08 15:15:35,132 - DEBUG - Transaction c0e145f8e648e029c9c85f345b5384d07bf0fffc5928e9d2e70a2e7910174896 

In [5]:
# resume all farms
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[StakingContract] = context.get_contracts(config.STAKINGS_V2)
for contract in contracts:
    tx_hash = contract.resume(context.deployer_account, context.network_provider.proxy)

2024-01-08 15:16:21,074 - DEBUG - Account.sync_nonce() done: 334
2024-01-08 15:16:21,075 - INFO - Resume stake contract
2024-01-08 15:16:21,076 - DEBUG - Calling resume at erd1qqqqqqqqqqqqqpgqsaryrfe84lqez2nvlv837krxnmp7zzgtexksjnynn3
2024-01-08 15:16:21,076 - DEBUG - Args: []
2024-01-08 15:16:21,465 - DEBUG - Contract call arguments: []
2024-01-08 15:16:21,645 - INFO - View this transaction in the MultiversX Testnet Explorer: https://testnet-explorer.multiversx.com/transactions/a6081a21e4bea5927643b09b65f1e1e1c7b78e68705a5da946856bd7df03a61e
2024-01-08 15:16:21,646 - INFO - Resume stake contract
2024-01-08 15:16:21,646 - DEBUG - Calling resume at erd1qqqqqqqqqqqqqpgqjmlxhn5njuw6g4vd6zdnhce74vxvejnaexks4wv4x6
2024-01-08 15:16:21,647 - DEBUG - Args: []
2024-01-08 15:16:21,857 - DEBUG - Contract call arguments: []
2024-01-08 15:16:22,083 - INFO - View this transaction in the MultiversX Testnet Explorer: https://testnet-explorer.multiversx.com/transactions/9329eac7146b9b35ea6536b545355b39

contract config

In [4]:
from time import sleep
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[StakingContract] = context.get_contracts(config.STAKINGS_V2)
simple_lock_energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
for contract in contracts:
    tx_hash = contract.set_boosted_yields_rewards_percentage(context.deployer_account, context.network_provider.proxy, 6000)
    tx_hash = contract.set_boosted_yields_factors(context.deployer_account, context.network_provider.proxy, 
                                                  [2, 1, 0, 1, 1])
    tx_hash = contract.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address)
    tx_hash = simple_lock_energy_contract.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, contract.address)
    sleep(2)

2023-12-19 12:56:40,763 - DEBUG - Account.sync_nonce() done: 3503
2023-12-19 12:56:40,764 - INFO - Set boosted yield rewards percentage
2023-12-19 12:56:40,765 - DEBUG - Arguments: [6000]
2023-12-19 12:56:40,765 - DEBUG - Calling setBoostedYieldsRewardsPercentage at erd1qqqqqqqqqqqqqpgqmqq78c5htmdnws8hm5u4suvags36eq092jpsaxv3e7
2023-12-19 12:56:40,766 - DEBUG - Args: [6000]
2023-12-19 12:56:41,774 - DEBUG - Contract call arguments: [6000]
2023-12-19 12:56:42,670 - INFO - No explorer known for https://proxy-shadowfork-one.elrond.ro. transaction raw path: transactions/eddf04ff80bd80f7ce5f612d5cd5dea8aebc033fe6c2084d20ce9b69701b599c
2023-12-19 12:56:42,671 - INFO - Set boosted yield factors
2023-12-19 12:56:42,672 - DEBUG - Arguments: [2, 1, 0, 1, 1]
2023-12-19 12:56:42,673 - DEBUG - Calling setBoostedYieldsFactors at erd1qqqqqqqqqqqqqpgqmqq78c5htmdnws8hm5u4suvags36eq092jpsaxv3e7
2023-12-19 12:56:42,673 - DEBUG - Args: [2, 1, 0, 1, 1]
2023-12-19 12:56:44,654 - DEBUG - Contract call argume

transfer tokens and prepare accounts

In [ ]:
staking_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, 1000000000)

In [ ]:
staking_contract.add_admin(context.deployer_account, context.network_provider.proxy, depositer.address.bech32())
staking_contract.topup_rewards(depositer, context.network_provider.proxy, 10000000000000000000)

In [ ]:
staking_contract.set_max_apr(context.deployer_account, context.network_provider.proxy, 10000)

In [ ]:
staking_contract.start_produce_rewards(context.deployer_account, context.network_provider.proxy)

In [ ]:
txhash = staking_contract.register_farm_token(context.deployer_account, context.network_provider.proxy, ["STKTST", "STKTST"])
status = context.network_provider.check_simple_tx_status(txhash, "set register farm token")

In [ ]:
status = context.network_provider.check_simple_tx_status("5f5e3b111bc5ca2f59e4f8bb8faf3aa7e7827bc0247808b53eb7bd58e9db19e4", "set boosted yields factors in farm")

In [ ]:
farm_token_hex = MetaStakingContractDataFetcher(Address(metastaking_contract.address), context.network_provider.proxy.url).get_data("getDualYieldTokenId")
print(hex_to_string(farm_token_hex))

enter stake

In [ ]:
event = EnterFarmEvent(staking_contract.farming_token, 0, 1000000000000000000, "", 0, 0)
txhash = staking_contract.stake_farm(context.network_provider, depositer, event, True)

claim stake

In [ ]:
event = ClaimRewardsFarmEvent(100000000000000000, 10, "")
txhash = staking_contract.claim_rewards(context.network_provider, depositer, event)

In [ ]:
from utils.utils_chain import get_token_details_for_address

farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(staking_contract.farm_token, depositer.address.bech32(), context.network_provider.proxy)
event = ClaimRewardsFarmEvent(farm_token_amount, farm_token_nonce, "")
txhash = staking_contract.claim_rewards(context.network_provider, depositer, event)

exit stake

In [ ]:
event = ExitFarmEvent(staking_contract.farm_token, 1000000, 9, "", 1000000)
txhash = staking_contract.unstake_farm(context.network_provider, depositer, event)

In [ ]:
from utils.utils_chain import get_token_details_for_address

farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(staking_contract.farm_token, depositer.address.bech32(), context.network_provider)
event = ClaimRewardsFarmEvent(farm_token_amount, farm_token_nonce, "")
txhash = staking_contract.claim_rewards(context.network_provider, depositer, event)

report contract data

In [ ]:
staking_tracker.update_data()
staking_tracker.report_current_tracking_data()

Shadowfork control

In [ ]:
from contracts.builtin_contracts import SFControlContract

sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)
sf_control_contract.epoch_fast_forward(context.deployer_account, context.network_provider.proxy, 16, 20)